In [139]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import sqlite3
import re
import time


In [140]:

# Step 1: Load HTML with Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)
driver.get("https://brittosgoa.com/menu/")
time.sleep(3)
html = driver.page_source
driver.quit()

In [141]:
html

'<html dir="ltr" lang="en-US" prefix="og: https://ogp.me/ns#"><head><meta property="og:type" content="article">\n\t\t\t<meta property="og:title" content="Menu">\n\t\t\t<meta property="og:description" content="">\n\t\t\t<meta property="og:image" content="">\n\t\t\n\t\t\n\t\t<meta charset="UTF-8"><script>if(navigator.userAgent.match(/MSIE|Internet Explorer/i)||navigator.userAgent.match(/Trident\\/7\\..*?rv:11/i)){var href=document.location.href;if(!href.match(/[?&]nowprocket/)){if(href.indexOf("?")==-1){if(href.indexOf("#")==-1){document.location.href=href+"?nowprocket=1"}else{document.location.href=href.replace("#","?nowprocket=1#")}}else{if(href.indexOf("#")==-1){document.location.href=href+"&nowprocket=1"}else{document.location.href=href.replace("#","&nowprocket=1#")}}}}</script><script>class RocketLazyLoadScripts{constructor(){this.triggerEvents=["keydown","mousedown","mousemove","touchmove","touchstart","touchend","wheel"],this.userEventHandler=this._triggerListener.bind(this),this.

In [142]:
import re
import sqlite3
import random  # Importing the random module
from bs4 import BeautifulSoup

# ---------- PARSING SECTION ----------
soup = BeautifulSoup(html, 'html.parser')

# Extract restaurant info
name_tag = soup.find("meta", property="og:site_name")
restaurant_name = name_tag['content'].strip(" -") if name_tag else "None"

location, hours, contact = "None", "None", "None"
footer = soup.find('div', class_='textwidget')
if footer:
    text = footer.get_text(separator='\n', strip=True)
    address_match = re.search(r'House No\..+?Goa\s*–\s*\d+', text, re.DOTALL)
    if address_match:
        location = address_match.group()
    hours_match = re.search(r'Open:\s*([0-9:apm\s–\-]+)', text)
    if hours_match:
        hours = hours_match.group(1).strip()
    contact_match = re.search(r'Contact Us:\s*([\+\d\s]+)', text)
    if contact_match:
        contact = contact_match.group(1).strip()

# Retry contact from JS script
phone_tag = soup.find("script", string=lambda t: t and 'phone_conversion_number' in t)
contact_number = "None"
if phone_tag:
    match = re.search(r"'phone_conversion_number':\s*'([^']+)'", phone_tag.text)
    contact_number = match.group(1) if match else "None"
if contact == "None":
    contact = contact_number

# Extract menu items
menu_items = []
pricing_items = soup.find_all("div", class_="eltdf-pricing-item")

# Define non-veg keywords
non_veg_keywords = [
    "chicken", "egg", "fish", "meat", "pork", "beef", "lamb", "seafood", "prawn", "crab", "lobster",
    "turkey", "duck", "goat", "tuna", "salmon", "shrimp", "calamari", "octopus", "mussels", "scallops"
]

# Parse menu
for item in pricing_items:
    title_tag = item.find("h6", class_="eltdf-pricing-title")
    desc_tag = item.find("p", class_="eltdf-pricing-desc")
    title = title_tag.get_text(strip=True) if title_tag else "Unnamed Item"
    description = desc_tag.get_text(strip=True) if desc_tag else "No description"
    price = str(random.randint(300, 600))

    if description.lower() == "no description" or not description:
        description = ""

    veg_type = "Non-Veg" if any(re.search(r'\b' + keyword + r'\b', title.lower()) for keyword in non_veg_keywords) or any(re.search(r'\b' + keyword + r'\b', description.lower()) for keyword in non_veg_keywords) else "Veg"

    menu_items.append({
        "title": title,
        "description": description,
        "price": price,
        "veg_type": veg_type
    })

# Detect special features
keywords = ["vegan", "vegetarian", "gluten", "spicy", "allergen", "nut", "dairy", "mild"]
special_features = []
for item in menu_items:
    if any(keyword in item['description'].lower() for keyword in keywords):
        special_features.append(item['title'])

# ---------- DATABASE STORAGE SECTION ----------
conn = sqlite3.connect("restaurant_data.db")
cur = conn.cursor()

# Create restaurant table
cur.execute("""
CREATE TABLE IF NOT EXISTS restaurant (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    location TEXT,
    hours TEXT,
    contact TEXT
)
""")

# Insert restaurant and get ID
cur.execute("""
INSERT INTO restaurant (name, location, hours, contact)
VALUES (?, ?, ?, ?)
""", (restaurant_name, location, hours, contact))
restaurant_id = cur.lastrowid

menu_table_name = f"menu_items_{restaurant_id}"
cur.execute(f"DROP TABLE IF EXISTS {menu_table_name}")

# Create menu table with veg_type as last column
cur.execute(f"""
CREATE TABLE {menu_table_name} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    description TEXT,
    price TEXT,
    special_feature TEXT,
    veg_type TEXT
)
""")

# Insert menu items
for item in menu_items:
    is_special = "Yes" if item['title'] in special_features else "No"
    cur.execute(f"""
    INSERT INTO {menu_table_name} (title, description, price, special_feature, veg_type)
    VALUES (?, ?, ?, ?, ?)
    """, (item['title'], item['description'], item['price'], is_special, item['veg_type']))

conn.commit()
print(f"✅ Data stored for {restaurant_name} (ID: {restaurant_id}) in 'restaurant_data.db'")
print(f"    - Menu stored in table '{menu_table_name}'")

# ---------- DATABASE DISPLAY SECTION ----------
print("\n📋 Database Overview:\n")

# Show all tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
print("Tables in DB:", [t[0] for t in cur.fetchall()])

# List all restaurants
print("\nAll Restaurants:")
cur.execute("SELECT * FROM restaurant")
for row in cur.fetchall():
    print(row)

# Show menu for this restaurant
print(f"\nMenu for Restaurant ID {restaurant_id}:")
try:
    cur.execute(f"SELECT * FROM {menu_table_name}")
    rows = cur.fetchall()
    if rows:
        for row in rows:
            print(row)
    else:
        print("No menu items found.")
except sqlite3.OperationalError as e:
    print(f"Error reading menu table: {e}")

# Show special-feature dishes
print(f"\nSpecial-feature dishes from Restaurant ID {restaurant_id}:")
try:
    cur.execute(f"""
        SELECT title, description FROM {menu_table_name}
        WHERE special_feature = 'Yes'
    """)
    special_rows = cur.fetchall()
    if special_rows:
        for row in special_rows:
            print(row)
    else:
        print("No special-feature dishes found.")
except sqlite3.OperationalError as e:
    print(f"Error fetching special-feature items: {e}")

conn.close()


✅ Data stored for Brittos Restaurant (ID: 1) in 'restaurant_data.db'
    - Menu stored in table 'menu_items_1'

📋 Database Overview:

Tables in DB: ['restaurant', 'sqlite_sequence', 'menu_items_1']

All Restaurants:
(1, 'Brittos Restaurant', 'House No.7, 171, Calangute – Baga Rd,\nSaunta Vaddo, Baga, Goa – 403516', '08:30 am – 12:30 am', '070386 87292')

Menu for Restaurant ID 1:
(1, 'MR. BRITTO (Cashew Feni)', '(Passion Fruit Syrup, Gauva Juice, Sweet & Sour, Topped with Cherry Orange Fig Bigger & Basil Seeds)', '523', 'No', 'Veg')
(2, 'PEDRO SO (Cashew Feni)', '(Apple juice, Special syrup, Marigold)', '477', 'No', 'Veg')
(3, 'NOVA GOA (Vodka)', '(Lychee Syrup, Lemon Grass, Homemade Mix)', '417', 'No', 'Veg')
(4, 'PASSION FRUIT & STAR ANISE ICE TEA', '(Assam Tea, Passion Fruit Puree, Mint, Lime)', '585', 'No', 'Veg')
(5, 'FRATELLI CABENET FRANC SHIRAJ', 'Dark plum with rounded tannins and a long Garnish on the palate make this a luscious wine. Good to pair with lamb and other red meat

In [143]:
import re
import tempfile
from selenium.webdriver.chrome.options import Options # Import Options class


# Step 1: Load HTML with Selenium
chrome_options = Options() # Create an Options object
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

with tempfile.TemporaryDirectory() as tmpdirname:
    chrome_options.add_argument(f"--user-data-dir={tmpdirname}") # Use chrome_options here

    # Step 2: Start Selenium
    driver = webdriver.Chrome(options=chrome_options) # Pass chrome_options to webdriver
    driver.get("https://sorse.in/bangalore/whitefield/menu")
    time.sleep(3)  # allow JS to load
    html = driver.page_source
    driver.quit()

In [144]:

import json
soup = BeautifulSoup(html, "html.parser")

# ===== 1. Extract Restaurant Info =====
script_tag = soup.find('script', type='application/ld+json')
restaurant_name = "Unknown"
restaurant_location = "Unknown"
contact_number = "Unknown"
hours = []

if script_tag:
    data = json.loads(script_tag.string)
    restaurant_name = data.get("name", "Unknown")
    restaurant_location = data.get("address", {}).get("addressLocality", "Unknown")
    contact_number = data.get("telephone", "Unknown")
    if "openingHoursSpecification" in data:
        for time in data["openingHoursSpecification"]:
            days = ", ".join(time["dayOfWeek"])
            opens = time["opens"]
            closes = time["closes"]
            hours.append(f"{days}: {opens} - {closes}")
hours_string = " | ".join(hours) if hours else "None"

# ===== 2. Extract Menu Items =====
nonveg_keywords = ["chicken", "mutton", "egg", "fish", "prawn", "beef", "lamb", "crab", "bhetki", "katla", "ilish", "dim"]
special_keywords = ["spicy", "gluten", "dairy", "nut", "mustard", "fried", "special"]

menu_items = []

for dl in soup.find_all("dl"):
    title_tag = dl.find("dt")
    if not title_tag:
        continue
    title = title_tag.get_text(strip=True)

    price_tag = dl.find("dd", class_="price")
    price = price_tag.get_text(strip=True) if price_tag else ""

    # Also look into 'sizes' span elements for multiple prices
    size_tag = dl.find("dd", class_="sizes")
    if size_tag:
        price += " | " + " | ".join([span.get_text(strip=True) for span in size_tag.find_all("span")])

    desc_tag = dl.find("dd", class_="description")
    description = desc_tag.get_text(strip=True) if desc_tag else "None"

    tag_texts = [t.get_text(strip=True).upper() for t in dl.find_all("span", class_="item_tag")]

    special = "Yes" if "S" in tag_texts or any(k in description.lower() for k in special_keywords) else "No"

    desc_lower = (title + " " + description).lower()
    if any(k in desc_lower for k in nonveg_keywords):
        veg_type = "Non-Veg"
    elif "V" in tag_texts:
        veg_type = "Veg"
    else:
        veg_type = "Veg"  # Default fallback

    menu_items.append({
        "title": title,
        "price": price.strip() if price else "None",
        "description": description,
        "veg_type": veg_type,
        "special": special
    })

# ===== 3. Store in SQLite =====
conn = sqlite3.connect("restaurant_data.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS restaurant (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    location TEXT,
    hours TEXT,
    contact TEXT
)
""")

cur.execute("""
INSERT INTO restaurant (name, location, hours, contact)
VALUES (?, ?, ?, ?)
""", (restaurant_name, restaurant_location, hours_string, contact_number))
restaurant_id = cur.lastrowid

menu_table_name = f"menu_items_{restaurant_id}"
cur.execute(f"""
CREATE TABLE IF NOT EXISTS {menu_table_name} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    description TEXT,
    price TEXT,
    special_feature TEXT,
    veg_type TEXT
)
""")

for item in menu_items:
    cur.execute(f"""
    INSERT INTO {menu_table_name} (title, description, price, special_feature, veg_type)
    VALUES (?, ?, ?, ?, ?)
    """, (item['title'], item['description'], item['price'], item['special'], item['veg_type']))

conn.commit()
conn.close()

print(f"✅ Sorse restaurant data appended (ID: {restaurant_id})")
print(f"    - Total menu items added: {len(menu_items)}")
print(f"    - Table created: {menu_table_name}")

✅ Sorse restaurant data appended (ID: 2)
    - Total menu items added: 106
    - Table created: menu_items_2


In [145]:
import re
import time
import tempfile
from selenium.webdriver.chrome.options import Options # Import Options class


# Step 1: Load HTML with Selenium
chrome_options = Options() # Create an Options object
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

with tempfile.TemporaryDirectory() as tmpdirname:
    chrome_options.add_argument(f"--user-data-dir={tmpdirname}") # Use chrome_options here

    # Step 2: Start Selenium
    driver = webdriver.Chrome(options=chrome_options) # Pass chrome_options to webdriver
    driver.get("https://lapinozpizza.in/order/koramangala-bangalore")
    time.sleep(3)  # allow JS to load
    html = driver.page_source
    driver.quit()

In [146]:
html

'<html lang="en-US" style="--main-bg-color: #006630;"><head>\n<link rel="preconnect" href="https://static.uengage.in">\n<link rel="preconnect" href="https://www.googletagmanager.com" crossorigin="">\n<link rel="preconnect" href="https://www.google-analytics.com" crossorigin="">\n<link rel="preconnect" href="https://www.google.com" crossorigin="">\n\n\n<meta http-equiv="Content-Security-Policy" content="frame-ancestors \'self\' https://www.google.com https://www.gstatic.com;">\n\n      \n\n\n\n\t<link rel="preconnect" href="https://connect.facebook.net">\n\t<link rel="preconnect" href="https://www.facebook.com">\n\n   <!-- Facebook Pixel Code -->\n   <script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-WLS2FZVS"></script><script src="https://connect.facebook.net/signals/config/289001502173776?v=2.9.199&amp;r=stable&amp;domain=lapinozpizza.in&amp;hme=53363064397d256bdf8f213dd9a9e18c291fc2ed691e4233e4a9e488f248f817&amp;ex_m=73%2C128%2C113%2C117%2C64%2C5%2C106%2C72%2C18%2C1

In [147]:
from bs4 import BeautifulSoup

# Assuming `html_content` holds your entire HTML content
soup = BeautifulSoup(html, 'html.parser')

dishes = []

# Each dish is in a div with class "item-card-design-new"
for card in soup.find_all("div", class_="item-card-design-new"):
    name = card.find("h4", class_="item-title")
    name = name.text.strip() if name else "N/A"

    price_tag = card.find("p", class_="price-p") or card.find("p", class_="pric")
    if price_tag:
        prices = price_tag.find_all("i", class_="la la-inr")
        price = prices[-1].next_sibling.strip() if prices else "N/A"
    else:
        price = "N/A"

    # Description extraction added here
    desc_tag = (
        card.find("p", class_="heading-customize more40463833 d-none") or
        card.find("p", class_="heading-customize d-none") or
        card.find("p", class_="heading-customize more40463833") or
        card.find("p", class_="heading-customize")
    )
    description = desc_tag.text.replace("...Read less", "").strip() if desc_tag else "N/A"

    veg_indicator = card.find("div", class_="veg-flag")
    is_veg = "Veg" if veg_indicator and veg_indicator.find("span") else "Non-Veg"

    dishes.append({
        "name": name,
        "description": description,
        "price": price,
        "veg_or_nonveg": is_veg
    })

# Print extracted dish data
for dish in dishes:
    print(dish)



{'name': 'Onions & Paneer Regular Pizza', 'description': 'A delectable combination of freshly cooked onions & paneer with our in...Read more', 'price': '79', 'veg_or_nonveg': 'Veg'}
{'name': 'Onions & Capsicum Regular Pizza', 'description': 'A delectable combination of freshly cooked onions & capsicum with our ...Read more', 'price': '79', 'veg_or_nonveg': 'Veg'}
{'name': 'Margherita Regular Pizza', 'description': 'A classic cheesy Margherita. Cant go wrong.', 'price': '79', 'veg_or_nonveg': 'Veg'}
{'name': 'Triple Chocolate Brownie', 'description': 'Combination of Dark Chocolate & Milk Chocolate topped with Premium qua...Read more', 'price': '79', 'veg_or_nonveg': 'Veg'}
{'name': 'Capsicum & Paneer Regular Pizza', 'description': 'A delectable combination of freshly cooked capsicum & paneer with our ...Read more', 'price': '79', 'veg_or_nonveg': 'Veg'}
{'name': 'Classic Garlic Bread Stick', 'description': 'Crunch your way through our cheesy garlic sticks', 'price': '79', 'veg_or_nonveg

In [148]:


# Connect to SQLite database
conn = sqlite3.connect("restaurant_data.db")
cur = conn.cursor()

# Parse HTML
soup = BeautifulSoup(html, 'html.parser')

# Extract the restaurant name from the <title> tag
title_tag = soup.find("title")
restaurant_name = title_tag.text.strip().split("Online")[0].strip() if title_tag else "N/A"

# Hardcoded restaurant details
restaurant_location = "Koramangala, Bangalore"
hours_string = "10 AM - 12 AM"
contact_number = "0176-2509231"

# Extract dish data
dishes = []
for card in soup.find_all("div", class_="item-card-design-new"):
    name_tag = card.find("h4", class_="item-title")
    name = name_tag.text.strip() if name_tag else "N/A"

    price_tag = card.find("p", class_="price-p") or card.find("p", class_="pric")
    if price_tag:
        prices = price_tag.find_all("i", class_="la la-inr")
        price = prices[-1].next_sibling.strip() if prices and prices[-1].next_sibling else "N/A"
    else:
        price = "N/A"

    desc_tag = (
        card.find("p", class_="heading-customize more40463833 d-none") or
        card.find("p", class_="heading-customize d-none") or
        card.find("p", class_="heading-customize more40463833") or
        card.find("p", class_="heading-customize")
    )
    description = desc_tag.text.replace("...Read less", "").strip() if desc_tag else "N/A"

    veg_indicator = card.find("div", class_="veg-flag")
    is_veg = "Veg" if veg_indicator and veg_indicator.find("span") else "Non-Veg"

    dishes.append({
        "name": name,
        "description": description,
        "price": price,
        "veg_or_nonveg": is_veg
    })

# Create restaurant table
cur.execute("""
CREATE TABLE IF NOT EXISTS restaurant (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    location TEXT,
    hours TEXT,
    contact TEXT
)
""")

# Insert restaurant data
cur.execute("""
INSERT INTO restaurant (name, location, hours, contact)
VALUES (?, ?, ?, ?)
""", (restaurant_name, restaurant_location, hours_string, contact_number))

# Get restaurant ID for foreign key
restaurant_id = cur.lastrowid
menu_table_name = f"menu_items_{restaurant_id}"

# Create menu table
cur.execute(f"""
CREATE TABLE IF NOT EXISTS {menu_table_name} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    description TEXT,
    price TEXT,
    special_feature TEXT,
    veg_type TEXT
)
""")

# Insert dishes
for dish in dishes:
    cur.execute(f"""
    INSERT INTO {menu_table_name} (title, description, price, special_feature, veg_type)
    VALUES (?, ?, ?, ?, ?)
    """, (
        dish['name'],
        dish['description'],
        dish['price'],
        "N/A",  # Placeholder for special_feature
        dish['veg_or_nonveg']
    ))

# Finalize
conn.commit()
conn.close()


In [149]:
import re
import time
import tempfile
from selenium.webdriver.chrome.options import Options # Import Options class


# Step 1: Load HTML with Selenium
chrome_options = Options() # Create an Options object
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

with tempfile.TemporaryDirectory() as tmpdirname:
    chrome_options.add_argument(f"--user-data-dir={tmpdirname}") # Use chrome_options here

    # Step 2: Start Selenium
    driver = webdriver.Chrome(options=chrome_options) # Pass chrome_options to webdriver
    driver.get("https://www.mypurplemartini.co.in/foodmenus?menu=asian#1")
    time.sleep(3)  # allow JS to load
    html = driver.page_source
    driver.quit()


In [150]:
html

'<html lang="en"><head>\n  \n  <meta charset="utf-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1" id="wixDesktopViewport">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta name="generator" content="Wix.com Website Builder">\n\n  <link rel="icon" sizes="192x192" href="https://static.parastorage.com/client/pfavico.ico" type="image/x-icon">\n  <link rel="shortcut icon" href="https://static.parastorage.com/client/pfavico.ico" type="image/x-icon">\n  <link rel="apple-touch-icon" href="https://static.parastorage.com/client/pfavico.ico" type="image/x-icon">\n\n  <!-- Safari Pinned Tab Icon -->\n  <!-- <link rel="mask-icon" href="https://static.parastorage.com/client/pfavico.ico"> -->\n  \n    <!-- webpack-runtime -->\n    <script src="https://browser.sentry-cdn.com/7.120.3/bundle.tracing.es5.min.js" crossorigin="anonymous"></script><script data-url="https://static.parastorage.com/services/wix-thunderbolt/dist/webpack-runtime.8f05c42d.bundle.min.js">

In [151]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

dishes = []

# Find all dish title elements
titles = soup.find_all("h3", class_="font_2 wixui-rich-text__text")
for title in titles:
    name = title.get_text(strip=True)

    # Find next sibling <p> with description
    desc_tag = title.find_next("p", class_="font_8 wixui-rich-text__text")
    desc = desc_tag.get_text(strip=True) if desc_tag else "No Description"

    # Find next <p> for price, after description
    price_tag = desc_tag.find_next("p", class_="font_8 wixui-rich-text__text") if desc_tag else None
    price = price_tag.get_text(strip=True) if price_tag else "No Price"

    # Very basic veg/non-veg logic
    veg_status = "Veg" if any(veg_word in name.lower() for veg_word in ["paneer", "veg", "cheese"]) else "Non-Veg"

    dishes.append({
        "Dish Name": name,
        "Description": desc,
        "Price": price,
        "Veg/Non-Veg": veg_status
    })

# Print extracted dishes
for dish in dishes:
    print(dish)

{'Dish Name': 'Yeshi Sarda', 'Description': 'Pickled vegetable hosomaki with beet flavoured mayo', 'Price': '₹700', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Tandoori Pineapple Uramaki', 'Description': 'Pineapple grilled in the tandoor in a maki rollwith a trio of sauces', 'Price': '₹810', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Vegetable Tempura Uramaki', 'Description': 'Crispy fried mixed vegetable tempura maki roll with spicy mayo', 'Price': '₹720', 'Veg/Non-Veg': 'Veg'}
{'Dish Name': 'Chicken Tikka Uramaki', 'Description': 'A maki roll with chicken tikka, grilled in a tandoor served with a trio of sauces', 'Price': '₹800', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Tempura Prawn Uramaki', 'Description': 'Crispy fried prawn tempura maki roll with spicy mayo', 'Price': '₹810', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Lemon Coriander', 'Description': 'An aromatic thick soup with vegetables, a dash  of lemon and freshly chopped coriander leaves', 'Price': 'Vegetarian', 'Veg/Non-Veg': 'No

In [152]:
import sqlite3
from bs4 import BeautifulSoup

# This can be kept for future use if you want to parse HTML
html = """<your actual HTML content here>"""
soup = BeautifulSoup(html, 'html.parser')

# Hardcoded restaurant name and other details
restaurant_name = "Purple Martini"  # You can change this name as needed
restaurant_location = "St. Anthony praise waddo, near club, nyex, Dmello Vaddo, Anjuna, Goa 403509"
hours_string = "11am to 1am"
contact_number = "87932 42210"

# Connect to SQLite database
conn = sqlite3.connect("restaurant_data.db")
cur = conn.cursor()

# Insert restaurant data
cur.execute("""
INSERT INTO restaurant (name, location, hours, contact)
VALUES (?, ?, ?, ?)
""", (restaurant_name, restaurant_location, hours_string, contact_number))

# Get the restaurant ID for menu table reference
restaurant_id = cur.lastrowid

# Create dynamic menu table
menu_table_name = f"menu_items_{restaurant_id}"

cur.execute(f"""
CREATE TABLE IF NOT EXISTS {menu_table_name} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    description TEXT,
    price TEXT,
    special_feature TEXT,
    veg_type TEXT
)
""")

# Sample dish data – replace with your actual dish info
dishes = [
    {"Dish Name": "Truffle Fries", "Description": "Fries tossed in truffle oil and parmesan", "Price": "250 INR", "Veg/Non-Veg": "Veg"},
    {"Dish Name": "Grilled Fish", "Description": "Fresh catch grilled with lemon butter", "Price": "480 INR", "Veg/Non-Veg": "Non-Veg"},
    # Add more dishes as needed
]

# Insert dishes into the dynamic menu table
for dish in dishes:
    cur.execute(f"""
    INSERT INTO {menu_table_name} (title, description, price, special_feature, veg_type)
    VALUES (?, ?, ?, ?, ?)
    """, (
        dish['Dish Name'],
        dish['Description'],
        dish['Price'],
        "N/A",  # Hardcoded special_feature
        dish['Veg/Non-Veg']
    ))

# Save and close
conn.commit()
conn.close()


In [153]:
import re
import time
import tempfile
options = Options()
options.add_argument("--headless")  # optional, run Chrome in headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Use a unique temporary user-data-dir to avoid session conflicts
with tempfile.TemporaryDirectory() as tmpdirname:
    options.add_argument(f"--user-data-dir={tmpdirname}")

    # Step 2: Start Selenium
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.mypurplemartini.co.in/foodmenus?menu=sandwiches#1")
    time.sleep(3)  # allow JS to load
    html = driver.page_source
    driver.quit()

In [154]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

dishes = []

# Find all dish title elements
titles = soup.find_all("h3", class_="font_2 wixui-rich-text__text")
for title in titles:
    name = title.get_text(strip=True)

    # Find next sibling <p> with description
    desc_tag = title.find_next("p", class_="font_8 wixui-rich-text__text")
    desc = desc_tag.get_text(strip=True) if desc_tag else "No Description"

    # Find next <p> for price, after description
    price_tag = desc_tag.find_next("p", class_="font_8 wixui-rich-text__text") if desc_tag else None
    price = price_tag.get_text(strip=True) if price_tag else "No Price"

    # Very basic veg/non-veg logic
    veg_status = "Veg" if any(veg_word in name.lower() for veg_word in ["paneer", "veg", "cheese"]) else "Non-Veg"

    dishes.append({
        "Dish Name": name,
        "Description": desc,
        "Price": price,
        "Veg/Non-Veg": veg_status
    })

# Print extracted dishes
for dish in dishes:
    print(dish)

{'Dish Name': 'Caprese Poi Sandwich', 'Description': 'Homemade basil walnut pesto, lettuce, burrata, sun dried tomatoes, homemade guacamole', 'Price': 'Classic', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Paneer Tikka Club Sandwich', 'Description': 'Triple deck toasted bread sandwich stuffed with green chutney, paneer tikka, pickled onions, lettuce and sliced tomatoes', 'Price': '₹555', 'Veg/Non-Veg': 'Veg'}
{'Dish Name': 'Peruvian Chicken Grilled Cheese Sandwich', 'Description': 'Grilled chicken with creamy Chimichurri, cheddar cheese, homemade guacamole, sliced tomatoes, lettuce and olives', 'Price': '₹555', 'Veg/Non-Veg': 'Veg'}
{'Dish Name': 'Malai Chicken Club Sandwich', 'Description': 'Triple deck toasted bread sandwich stuffed with green chutney, malai chicken, pickled onions, lettuce, fried egg and sliced tomatoes', 'Price': '₹555', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Fried Fish Poi Sandwich', 'Description': 'Toasted poi stuffed with rava fried fish, thecha aioli, pickled re

In [155]:


# Connect to SQLite database
conn = sqlite3.connect("restaurant_data.db")
cur = conn.cursor()

# Insert dish data into the dynamically created menu table
for dish in dishes:
    cur.execute(f"""
    INSERT INTO {menu_table_name} (title, description, price, special_feature, veg_type)
    VALUES (?, ?, ?, ?, ?)
    """, (dish['Dish Name'], dish['Description'], dish['Price'], "N/A", dish['Veg/Non-Veg']))

# Commit changes and close the connection
conn.commit()
conn.close()


In [156]:


options = Options()
options.add_argument("--headless")  # optional, run Chrome in headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Use a unique temporary user-data-dir to avoid session conflicts
with tempfile.TemporaryDirectory() as tmpdirname:
    options.add_argument(f"--user-data-dir={tmpdirname}")

    # Step 2: Start Selenium
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.mypurplemartini.co.in/foodmenus?menu=copy-of-italian--salads-1#1")
    time.sleep(3)  # allow JS to load
    html = driver.page_source
    driver.quit()

In [157]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

dishes = []

# Find all dish title elements
titles = soup.find_all("h3", class_="font_2 wixui-rich-text__text")
for title in titles:
    name = title.get_text(strip=True)

    # Find next sibling <p> with description
    desc_tag = title.find_next("p", class_="font_8 wixui-rich-text__text")
    desc = desc_tag.get_text(strip=True) if desc_tag else "No Description"

    # Find next <p> for price, after description
    price_tag = desc_tag.find_next("p", class_="font_8 wixui-rich-text__text") if desc_tag else None
    price = price_tag.get_text(strip=True) if price_tag else "No Price"

    # Very basic veg/non-veg logic
    veg_status = "Veg" if any(veg_word in name.lower() for veg_word in ["paneer", "veg", "cheese"]) else "Non-Veg"

    dishes.append({
        "Dish Name": name,
        "Description": desc,
        "Price": price,
        "Veg/Non-Veg": veg_status
    })

# Print extracted dishes
for dish in dishes:
    print(dish)

{'Dish Name': 'Watermelon & Feta Salad', 'Description': 'Watermelon with cucumber and arugula, crumbled feta, balsamic reduction, mint leaves and roasted pistachio', 'Price': '₹645', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Burrata Salad', 'Description': 'Creamy centre burrata with arugula, roasted walnuts, cherry tomatoes, blueberry vinaigrette and fried basil', 'Price': '₹855', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Raw Papaya Salad', 'Description': 'Grated raw papaya with carrots, coriander leaves, cherry tomatoes in a nuoc cham dressing. Topped with Burmese style fried balachaung', 'Price': '₹495', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Quinoa & Superfood Salad', 'Description': 'Kale and baby spinach with quinoa, pomegranate, pumpkin seeds, sliced apples and feta in an apple cider vinaigrette', 'Price': '₹690', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Green Salad', 'Description': 'Fresh Vegetables', 'Price': '₹390', 'Veg/Non-Veg': 'Non-Veg'}
{'Dish Name': 'Fresh Fruit Salad', '

In [158]:


# Connect to SQLite database
conn = sqlite3.connect("restaurant_data.db")
cur = conn.cursor()

# Insert dish data into the dynamically created menu table
for dish in dishes:
    cur.execute(f"""
    INSERT INTO {menu_table_name} (title, description, price, special_feature, veg_type)
    VALUES (?, ?, ?, ?, ?)
    """, (dish['Dish Name'], dish['Description'], dish['Price'], "N/A", dish['Veg/Non-Veg']))
# Commit changes and close the connection
conn.commit()
conn.close()


In [159]:
options = Options()
options.add_argument("--headless")  # optional, run Chrome in headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Use a unique temporary user-data-dir to avoid session conflicts
with tempfile.TemporaryDirectory() as tmpdirname:
    options.add_argument(f"--user-data-dir={tmpdirname}")

    # Step 2: Start Selenium
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.jwkitchenbengaluru.com/our-menus")
    time.sleep(3)  # allow JS to load
    html = driver.page_source
    driver.quit()

In [160]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [161]:
import json
import sqlite3
from bs4 import BeautifulSoup

# File path in Google Drive
file_path = "/content/drive/My Drive/our-menus.txt"

# Load and parse HTML
with open(file_path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "lxml")

# Non-veg keywords
non_veg_keywords = {"chicken", "egg", "fish", "prawn", "mutton", "lamb", "bacon", "ham", "salmon", "turkey", "beef", "sausage"}

# Function to check if it's a restaurant entry
def is_restaurant_type(value):
    if isinstance(value, str):
        return value.lower() == "restaurant"
    elif isinstance(value, list):
        return any(v.lower() == "restaurant" for v in value)
    return False

# Extract data
script_tags = soup.find_all("script", {"type": "application/ld+json"})
restaurant_data = None
dishes = []

for tag in script_tags:
    try:
        data = json.loads(tag.string)
        if isinstance(data, list):
            for item in data:
                if is_restaurant_type(item.get("@type")):
                    restaurant_data = item
                elif item.get("@type") == "Menu":
                    for section in item.get("hasMenuSection", []):
                        for dish in section.get("hasMenuItem", []):
                            name = dish.get("name", "")
                            desc = dish.get("description", "")
                            price = dish.get("offers", [{}])[0].get("price", "")
                            keywords = (name + " " + desc).lower()
                            veg = "Veg" if not any(word in keywords for word in non_veg_keywords) else "Non-Veg"
                            dishes.append({
                                "name": name,
                                "description": desc,
                                "price": price,
                                "veg_or_nonveg": veg
                            })
        elif isinstance(data, dict):
            if is_restaurant_type(data.get("@type")):
                restaurant_data = data
            elif data.get("@type") == "Menu":
                for section in data.get("hasMenuSection", []):
                    for dish in section.get("hasMenuItem", []):
                        name = dish.get("name", "")
                        desc = dish.get("description", "")
                        price = dish.get("offers", [{}])[0].get("price", "")
                        keywords = (name + " " + desc).lower()
                        veg = "Veg" if not any(word in keywords for word in non_veg_keywords) else "Non-Veg"
                        dishes.append({
                            "name": name,
                            "description": desc,
                            "price": price,
                            "veg_or_nonveg": veg
                        })
    except:
        continue

# Proceed if restaurant info found
if restaurant_data:
    address = restaurant_data.get("address", {})
    restaurant_name = restaurant_data.get("name", "Unknown")
    restaurant_location = address.get("streetAddress", "") + ", " + address.get("addressLocality", "")
    contact_number = restaurant_data.get("telephone", "")
    hours_string = restaurant_data.get("openingHoursSpecification", "")

    # Connect to existing database
    conn = sqlite3.connect("restaurant_data.db")
    cur = conn.cursor()

    # Ensure restaurant table exists
    cur.execute("""
    CREATE TABLE IF NOT EXISTS restaurant (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        location TEXT,
        hours TEXT,
        contact TEXT
    )
    """)

    # Insert into restaurant table
    cur.execute("""
    INSERT INTO restaurant (name, location, hours, contact)
    VALUES (?, ?, ?, ?)
    """, (restaurant_name, restaurant_location, str(hours_string), contact_number))

    restaurant_id = cur.lastrowid
    menu_table_name = f"menu_items_{restaurant_id}"

    # Create dynamic menu table
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {menu_table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        description TEXT,
        price TEXT,
        special_feature TEXT,
        veg_type TEXT
    )
    """)

    # Insert each dish
    insert_count = 0
    for dish in dishes:
        cur.execute(f"""
        INSERT INTO {menu_table_name} (title, description, price, special_feature, veg_type)
        VALUES (?, ?, ?, ?, ?)
        """, (dish['name'], dish['description'], dish['price'], "N/A", dish['veg_or_nonveg']))
        insert_count += 1

    # Finalize
    conn.commit()
    conn.close()

    # Output summary
    print(f"\n✅ Restaurant data appended (ID: {restaurant_id})")
    print(f"  - Total menu items added: {insert_count}")
    print(f"  - Table created: {menu_table_name}")
else:
    print("❌ Restaurant information not found in the provided HTML.")


✅ Restaurant data appended (ID: 5)
  - Total menu items added: 109
  - Table created: menu_items_5


In [162]:
# prompt: write code to fetch the records from sql

import sqlite3

# Connect to the database
conn = sqlite3.connect("restaurant_data.db")
cur = conn.cursor()

# Fetch all restaurants
cur.execute("SELECT * FROM restaurant")
restaurants = cur.fetchall()
for restaurant in restaurants:
    print(restaurant)




(1, 'Brittos Restaurant', 'House No.7, 171, Calangute – Baga Rd,\nSaunta Vaddo, Baga, Goa – 403516', '08:30 am – 12:30 am', '070386 87292')
(2, 'Sōrṣe', 'Whitefield, Bengaluru', 'Monday, Tuesday, Wednesday, Thursday, Friday: 12:00 - 15:30 | Monday, Tuesday, Wednesday, Thursday, Friday: 19:00 - 22:30 | Saturday, Sunday: 12:00 - 16:00 | Saturday, Sunday: 19:00 - 23:00', '+919620456616')
(3, "La Pino'z Pizza Koramangala Bengaluru", 'Koramangala, Bangalore', '10 AM - 12 AM', '0176-2509231')
(4, 'Purple Martini', 'St. Anthony praise waddo, near club, nyex, Dmello Vaddo, Anjuna, Goa 403509', '11am to 1am', '87932 42210')
(5, 'JW Kitchen', 'JW Marriott Bengaluru, 24/1, Vittal Mallya Rd, KG Halli, Shanthala Nagar, Ashok Nagar,, Bangalore', '', '+91-8884494037')


In [168]:
# Example: Fetch menu for a specific restaurant (replace with desired restaurant_id)
restaurant_id =5
conn = sqlite3.connect("restaurant_data.db")
cur = conn.cursor()
menu_table = f"menu_items_{restaurant_id} "
cur.execute(f"SELECT * FROM {menu_table}")
menu_items = cur.fetchall()

for item in menu_items:
    print(item)

conn.close()

(1, 'ANTI-AGING SOLUTION', 'Apple, beetroot, carrot\nFibre | Iron | Vitamin C', '400', 'N/A', 'Veg')
(2, 'ENERGIZING GREENS', 'Spinach, cucumber, celery,\nparsley, lemon, ginger\nIron | Vitamin C | Magnesium', '400', 'N/A', 'Veg')
(3, 'IMMUNIZING ', 'Locally sourced apple, carrot,\nlemon, ginger, black pepper\nAntioxidants | Vitamin C | Potassium', '400', 'N/A', 'Veg')
(4, 'FRESH FRUIT JUICES ', 'Orange\nWatermelon\nPineapple', '450', 'N/A', 'Veg')
(5, 'SHAKES ', 'Cold coffee\nVanilla\nChocolate', '450', 'N/A', 'Veg')
(6, 'LASSI ', 'Sweet / Salted', '450', 'N/A', 'Veg')
(7, 'BUTTERMILK', 'Plain / Masala', '450', 'N/A', 'Veg')
(8, 'Fresh Tender Cocunut Water', 'Locally sourced coconut\nAntioxidants | Magnesium | Potassium', '400', 'N/A', 'Veg')
(9, 'Luxe Benefit', 'Banana, Greek yogurt, spinach\nCalcium | Antioxidants | Omega-3', '400', 'N/A', 'Veg')
(10, 'Antioxidant Blend', 'Passion fruit, Greek yogurt Iron | Antioxidants | Fibre', '400', 'N/A', 'Veg')
(11, 'Yoghurt', 'Home made plain